In [ ]:
import pandas as pd
from finx_ib_reports.input_sources import TwsApiSource

from ib_insync import util, IB, Index, Future, Contract, Option

In [ ]:
util.startLoop()

In [ ]:
ib = IB()
ib.connect('127.0.0.1', 8888, clientId=5)

In [ ]:
crude = Future('CL', lastTradeDateOrContractMonth='202204', exchange='NYMEX', localSymbol="CLJ2")
ib.qualifyContracts(crude)
crude

In [ ]:
ib.reqMarketDataType(3)

In [ ]:
[ticker] = ib.reqTickers(crude)
print(ticker)

crude_current_price = ticker.marketPrice()
crude_current_price

In [ ]:
# chains = ib.reqSecDefOptParams(crude.symbol, crude.exchange, crude.secType, crude.conId)
# util.df(chains)

In [ ]:
def gen_option_contracts(contract):
    return Option(contract.localSymbol, "20220225", 100.0, "C", multiplier=contract.multiplier)

options = gen_option_contracts(ticker.contract)

In [ ]:
contract = ticker.contract
# option = Option(contract.localSymbol, "20220225", 100.0, "C", multiplier=contract.multiplier)

import numpy as np

def gen_fops_contracts(contract: Contract, expiration_date: str):
    contracts = []
    for strike in np.arange(96, 100, 0.5):
        for put_call in ["C"]:
            contract = Contract.create(
                secType="FOP", 
                conId=contract.conId, 
                symbol=contract.symbol, 
                lastTradeDateOrContractMonth=expiration_date,
                strike=100.0,
                includeExpired=False
            )
            contracts.append(contract)
    return contracts

contracts = gen_fops_contracts(ticker.contract, "20220225")
ib.qualifyContracts(*contracts)

len(contracts)

In [ ]:
tickers = ib.reqTickers(*contracts)

t = tickers[0]

ib.reqMktData(crude)
underlying_price = ticker.marketPrice()

# iv = ib.calculateImpliedVolatility(crude, t.bid, underlying_price, [])

# option = Option('AAPL', '20220225', 150, 'C', "SMART", multiplier=100)

# calc = ib.calculateImpliedVolatility(option, optionPrice=6.1, underPrice=150)
# print(calc)

# # calc = ib.calculateOptionPrice(
# #     option, volatility=0.14, underPrice=525))
# # print(calc)
# calc = ib.calculateOptionPrice(t.contract, volatility=0.51, underPrice=97)
# print(calc)

# x = ib.reqMktData(t.contract, "233", False, False)

# x = ib.reqMktData(t.contract, "12", False, False)

t.contract

In [ ]:
ib.disconnect()

In [ ]:
t.avOptionVolume